## Use `svgwrite` to get the logo

In [1]:
import svgwrite

## Get the object to draw

In [2]:
width = 36
height = 18

In [3]:
dwg = svgwrite.Drawing('ACMHomepage.svg', profile='tiny')

## Add paths to the group `main`

In [4]:
class MyPathCommand:
    def __init__(self, fun):
        self.fun = fun
        
    def exec(self, scale):
        unrounded_result = self.fun(scale)
        result = []
        for i in unrounded_result:
            if type(i) == float:
                result.append(round(i, 2))
            else:
                result.append(i)
        return tuple(result)

class MyPath:
    def __init__(self, value = [], scale = 1):
        self.value = []
        for i in value:
            self._add_command(lambda scale: i)
        self.scale = scale
        
    def _add_command(self, fun):
        self.value.append(MyPathCommand(fun))
        
    def set_scale(self, new_scale):
        new_path = MyPath()
        new_path.value = self.value
        new_path.scale = new_scale
        return new_path
        
    def v(self, length):
        self._add_command(lambda scale: ('v', length * scale))
        return self
        
    def h(self, length):
        self._add_command(lambda scale: ('h', length * scale))
        return self
        
    def l(self, x, y):
        self._add_command(lambda scale: ('l', x * scale, y * scale))
        return self
    
    def m(self, x, y):
        self._add_command(lambda scale: ('m', x * scale, y * scale))
        return self
        
    def ar(self, r, dir, x, y):
        self._add_command(lambda scale: ('a', r* scale, r * scale, 0, 0, dir, x * scale, y * scale))
        return self
    
    def z(self):
        self._add_command(lambda scale: ('z',))
        return self
    
    def cat(self, other):
        assert(self.scale == other.scale)
        self.value.extend(other.value)
        return self
    
    def get_path(self):
        result = []
        for i in self.value:
            result.append(i.exec(self.scale))
        return dwg.path(result)
    
    def __str__(self):
        return str(self.get_path().get_xml())

In [5]:
h1 = 1.2

v2 = 6
vh = 9

k1x = 2.8
k1y = 6
k2x = 1
k2y = 4

r2 = 2.6
R2 = r2 * 2
r1 = r2 - h1
R1 = r1 * 2

In [6]:
# util
up_half = MyPath().v(-(vh - h1) / 2)
down_half = MyPath().v((vh - h1) / 2)

# The A
A_path = MyPath().h(h1).l(k1x, -k1y).v(v2).h(h1).v(-vh).h(-h1).z();

# The C
C_path = (MyPath().m(0, -r2)
    .ar(r2, 0, R2, 0).h(-h1).ar(r1, 1, -R1, 0)
    .v(-vh + R2)
    .ar(r1, 1, R1, 0).h(h1).ar(r2, 0, -R2, 0)
    .z())

# The E
E_path = MyPath().h(-R2 + h1).v(-(vh - 3 * h1) / 2).h(R2 - h1).v(-h1)
E_path = MyPath().h(R2).v(-h1).cat(E_path).cat(E_path).h(-R2).z()

# The G
G_path = (MyPath()
    .m(0, -r2).ar(r2, 0, r2, r2).h(r2).v(-r2 - h1)
    .h(-h1).v(r2)
    .h(-r2 + h1).ar(r1, 1, -r1, -r1)
    .v(- vh + R2).ar(r1, 1, R1, 0)
    .h(h1).ar(r2, 0, -R2, 0).z())

# The H
H_path = (MyPath()
    .h(h1).cat(up_half).h(R2 -  2*h1).cat(down_half).h(h1)
    .v(-vh)
    .h(-h1).cat(down_half).h(-R2 +  2*h1).cat(up_half).h(-h1)
    .z())

# The M
M_path = MyPath().h(h1).l(k2x, -k2y).v(k2y)
M_path = M_path.cat(M_path).h(h1).v(-vh).h(-h1).l(-k2x, k2y).v(-k2y).h(-h1).z()

# The O
O_path = MyPath().m(0, -r2).ar(r2, 0, R2, 0).v(-vh + R2).ar(r2, 0, -R2, 0).z()

# The P
P_path = MyPath().h(h1).v(-vh + R2).h(r2 - h1).ar(r2, 0, 0, -R2).h(-r2).z()

In [12]:
sep = 1.5

main = dwg.g(stroke="currentColor", fill="currentColor", stroke_width='.4', stroke_linecap="round", stroke_linejoin="round")

x = 0
main.add(MyPath([('M', x, 9)]).cat(A_path).get_path())

x += 7.2
main.add(MyPath([('M', x, 9)]).cat(C_path).get_path())

x += (7.2 - 0.8)
main.add(MyPath([('M', x, 9)]).cat(M_path).get_path())

x = 0
main.add(MyPath([('M', x, vh * 1.7 + sep)], 0.7).cat(H_path.set_scale(0.7)).get_path())

x += 7.2 * 0.7
main.add(MyPath([('M', x, vh * 1.7 + sep)], 0.7).cat(O_path.set_scale(0.7)).get_path())

x += (7.2 - 0.8) * 0.7
main.add(MyPath([('M', x, vh * 1.7 + sep)], 0.7).cat(M_path.set_scale(0.7)).get_path())

x += (7.2 + 0.4) * 0.7
main.add(MyPath([('M', x, vh * 1.7 + sep)], 0.7).cat(E_path.set_scale(0.7)).get_path())

x += 7.2 * 0.7
main.add(MyPath([('M', x, vh * 1.7 + sep)], 0.7).cat(P_path.set_scale(0.7)).get_path())

x += (7.2 - 2.4) * 0.7
main.add(MyPath([('M', x, vh * 1.7 + sep)], 0.7).cat(A_path.set_scale(0.7)).get_path())

x += 7.2 * 0.7
main.add(MyPath([('M', x, vh * 1.7 + sep)], 0.7).cat(G_path.set_scale(0.7)).get_path())

x += 7.2 * 0.7
main.add(MyPath([('M', x, vh * 1.7 + sep)], 0.7).cat(E_path.set_scale(0.7)).get_path())

width = round(x + R2 * 0.7, 2)
height = round(vh * 1.7 + sep, 2)

## Add the two groups to the object, and show

In [13]:
dwg = svgwrite.Drawing('ACMHomepage.svg', profile='tiny')
dwg.viewbox(-1, -1, width + 2, height + 2)
dwg.add(main)

In [14]:
dwg

In [15]:
dwg.tostring()

'<svg baseProfile="tiny" height="100%" version="1.2" viewBox="-1,-1,38.96,18.8" width="100%" xmlns="http://www.w3.org/2000/svg" xmlns:ev="http://www.w3.org/2001/xml-events" xmlns:xlink="http://www.w3.org/1999/xlink"><defs /><g fill="currentColor" stroke="currentColor" stroke-linecap="round" stroke-linejoin="round" stroke-width=".4"><path d="M 0 9 h 1.2 l 2.8 -6 v 6 h 1.2 v -9 h -1.2 z" /><path d="M 7.2 9 m 0 -2.6 a 2.6 2.6 0 0 0 5.2 0 h -1.2 a 1.4 1.4 0 0 1 -2.8 0 v -3.8 a 1.4 1.4 0 0 1 2.8 0 h 1.2 a 2.6 2.6 0 0 0 -5.2 0 z" /><path d="M 13.6 9 h 1.2 l 1 -4 v 4 h 1.2 l 1 -4 v 4 h 1.2 v -9 h -1.2 l -1 4 v -4 h -1.2 z" /><path d="M 0 16.8 h 0.84 v -2.73 h 1.96 v 2.73 h 0.84 v -6.3 h -0.84 v 2.73 h -1.96 v -2.73 h -0.84 z" /><path d="M 5.04 16.8 m 0.0 -1.82 a 1.82 1.82 0 0 0 3.64 0.0 v -2.66 a 1.82 1.82 0 0 0 -3.64 0.0 z" /><path d="M 9.52 16.8 h 0.84 l 0.7 -2.8 v 2.8 h 0.84 l 0.7 -2.8 v 2.8 h 0.84 v -6.3 h -0.84 l -0.7 2.8 v -2.8 h -0.84 z" /><path d="M 14.84 16.8 h 3.64 v -0.84 h -2.8 v 

In [16]:
dwg.save()